In [1]:
import os
from configparser import ConfigParser
path_config_file = os.getcwd().split('docrv2_sroie')[0] + 'docrv2_sroie/' + 'config.ini'
common_variable = ConfigParser()
common_variable.read(path_config_file)

print(common_variable.sections())
print(common_variable['database']['module_path'])
print(common_variable['database']['DB_BASE'])

import sys
sys.path.append(common_variable['database']['module_path'])
from db_util import *
from db import *

import tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import time

from parse import parse
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
from glob import glob
from tqdm import tqdm_notebook

['database', 'mlflow', 'font']
/media/myungsungkwak/msdisk/docrv2_sroie/modules/database/
/media/myungsungkwak/msdisk/docrv2_data/DB/


In [71]:
"""
Evalution
"""
raw_db_name = "raw_recog_inference_data_all_0001" # change
raw_db = open_env(os.path.join(common_variable['database']['DB_BASE'], raw_db_name))
raw_refs = read_bulk_data_from_db(raw_db, prefix='ref')
# display(raw_refs)

target_datadb_name = "recog_inference_result_all_0001" # change
target_datadb_path = os.path.join(common_variable['database']['DB_BASE'], target_datadb_name)
target_datadb = open_env(path=target_datadb_path)
words = read_bulk_data_from_db(target_datadb, prefix='word')
# print(words)

raw_data_path = '/media/myungsungkwak/msdisk/docrv2_sroie/data/test_task12_img_text/'
txt_file_list = glob(raw_data_path + '*.txt')
output_path = '/media/myungsungkwak/msdisk/docrv2_sroie/evaluation/task2/inf_all_0907'
os.makedirs(output_path, exist_ok=True)

for idx in range(len(txt_file_list)):
    refname = raw_refs[idx]
    with open(os.path.join(output_path, refname+'.txt'), 'w') as f:
        for ii in range(len(words[idx])):
            for item in words[idx][ii].split():
                f.write(item)
                f.write('\n') 
print("...complete")

- refname :  X51005746206
- refname :  X51005675099
- refname :  X51005444046
- refname :  X51006557115
- refname :  X51008099100
- refname :  X51005568855
- refname :  X51006414717
- refname :  X51009008091
- refname :  X51006828200
- refname :  X51006619570
- refname :  X51006414728
- refname :  X51007339122
- refname :  X51005757308
- refname :  X51007391390
- refname :  X51005757342
- refname :  X51005711402
- refname :  X51006502534
- refname :  X51005719895
- refname :  X51005663310
- refname :  X51005568890
- refname :  X51006401845
- refname :  X51007231276
- refname :  X51006556740
- refname :  X51005442343
- refname :  X51005715007
- refname :  X51006857126
- refname :  X51006619704
- refname :  X51006647933
- refname :  X51005582745
- refname :  X51006556658
- refname :  X51006414600
- refname :  X51006619760
- refname :  X51009453729
- refname :  X51005447844
- refname :  X51006556826
- refname :  X51006334139
- refname :  X51005442322
- refname :  X51005764031
- refname : 

In [183]:
"""
Visualize - check 
"""
import json

raw_db_name = "raw_recog_inference_data_all_0001" # change
raw_db = open_env(os.path.join(common_variable['database']['DB_BASE'], raw_db_name))
raw_refs = read_bulk_data_from_db(raw_db, prefix='ref')
raw_char_c = read_bulk_data_from_db(raw_db, prefix='char_c')

inf_db_name = "recog_inference_result_all_0001" # change
inf_db_path = os.path.join(common_variable['database']['DB_BASE'], inf_db_name)
inf_db = open_env(path=inf_db_path)
inf_words = read_bulk_data_from_db(inf_db, prefix='word')
inf_words_i = read_bulk_data_from_db(inf_db, prefix='word_i')

json_path = '/media/myungsungkwak/msdisk/docrv2_sroie/evaluation/task2/result_0907/'
img_path = '/media/myungsungkwak/msdisk/docrv2_sroie/data/test_task12_img_text/'
out_path = '/media/myungsungkwak/msdisk/docrv2_data/task2_eval/'
os.makedirs(out_path, exist_ok=True)

right_font = ImageFont.truetype('/media/myungsungkwak/msdisk/docrv2_data/fonts/OpenSans-Regular.ttf', 10)
wrong_font = ImageFont.truetype('/media/myungsungkwak/msdisk/docrv2_data/fonts/OpenSans-Regular.ttf', 15)
# print(os.path.join(json_path, raw_refs[0]+'.json'))

max_ind = len(raw_refs)
for idx in tqdm(range(max_ind)):
#     print("... in progress : ", raw_refs[idx]+".jpg")
    with open(os.path.join(json_path, raw_refs[idx]+'.json')) as json_file:
        json_data = json.load(json_file)
        tt = "precision : {}\nrecall : {}\nhmean : {}".format(json_data['precision'],
                                                              json_data['recall'],
                                                              json_data['hmean']
                                                             )
        
        img = Image.open(os.path.join(img_path, raw_refs[idx]+".jpg"))
        draw_inf = ImageDraw.Draw(img)
        draw_inf.text((5, 5), tt, fill='black', font=wrong_font)
        
        for ii, coords in enumerate(raw_char_c[idx]):
            
            iw = inf_words[idx][ii].replace(" ","")
            rw = coords[8].replace(" ","")
            
#             print("[inf] {} {}".format(idx, inf_words[idx][ii].replace(" ","")))
#             print("[raw] {} {}".format(idx, coords[8].replace(" ","")))
            #print(inf_words[idx][ii])
            if iw == rw:
                # good (green)
#                 draw_inf.rectangle([int(coords[0]),
#                                     int(coords[1]),
#                                     int(coords[4]),
#                                     int(coords[5])], outline='green')
                draw_inf.text((int(coords[0]), int(coords[1])-11), coords[8], fill='green', font=right_font)
            else:
                # bad (red)
                draw_inf.rectangle([int(coords[0]),
                                    int(coords[1]),
                                    int(coords[4]),
                                    int(coords[5])], outline='red')
                #sw = coords[8] + "-> " + inf_words[idx][ii]
                #draw_inf.text((int(coords[0]), int(coords[1])-17), sw, fill='red', font=wrong_font)
                draw_inf.text((int(coords[0]), int(coords[1])-17), inf_words[idx][ii], fill='red', font=wrong_font)
    img.save(os.path.join(out_path,raw_refs[idx]+'.jpg'))
print("... complete")

100%|██████████| 361/361 [00:53<00:00,  6.77it/s]

... complete


In [154]:
def compare(inf_str, raw_str):
    diff_idx = []
    for idx, ch in enumerate(raw_str):
        if inf_str[idx] != ch:
            diff_idx.append(idx)
    return diff_idx

str1 = "NO.10-G,GHOUNDFLOOR." # inf
str2 = "NO.10-G.GROUNDFLOOR." # raw

diff_idx = compare(str1, str2)
print(diff_idx)

[7, 9]
